In [ ]:
import pandas as pd

In [ ]:
# function to read txt data files and convert them to proper csv files
# txtFile: input filename (including directory if applicable)
# csvFile: output filename (including directory if applicable)
# vtabchar: vertical tab character in the original file (to be replaced with newline command '\n')
# delim: delimiter character used in the original file (to be replaced with comma)
def txt2csv(txtFile, csvFile, vtabchar, delim):
    with open(txtFile, 'r') as file:
        data = file.read().replace(vtabchar, '\n').replace(delim, ',')
    with open(csvFile, 'w') as file:
        file.write(data)    
    return

Set the filename and location for each dataset

In [ ]:
# original filename and directory for txt data files
K1DI2_txt = './Data/Komponente/Komponente_K1DI2.txt'
K2LE1_txt = './Data/Komponente/Komponente_K2LE1.txt' 
K2LE2_txt = './Data/Komponente/Komponente_K2LE2.txt'
K2ST1_txt = './Data/Komponente/Komponente_K2ST1.txt'
K3AG2_txt = './Data/Komponente/Komponente_K3AG2.txt'
K7_txt    = './Data/Komponente/Komponente_K7.txt'

# converted txt filename and directory
K1DI2_csv = './Data/Komponente/Komponente_K1DI2.csv'
K2LE1_csv = './Data/Komponente/Komponente_K2LE1.csv'
K2LE2_csv = './Data/Komponente/Komponente_K2LE2.csv'
K2ST1_csv = './Data/Komponente/Komponente_K2ST1.csv'
K3AG2_csv = './Data/Komponente/Komponente_K3AG2.csv'
K7_csv    = './Data/Komponente/Komponente_K7.csv'

# original filename and directory for csv data files
# component data files
K1BE1_csv = './Data/Komponente/Komponente_K1BE1.csv'
K1BE2_csv = './Data/Komponente/Komponente_K1BE2.csv'
K1DI1_csv = './Data/Komponente/Komponente_K1DI1.csv'
K2ST2_csv = './Data/Komponente/Komponente_K2ST2.csv'
K3AG1_csv = './Data/Komponente/Komponente_K3AG1.csv'
K3SG1_csv = './Data/Komponente/Komponente_K3SG1.csv'
K3SG2_csv = './Data/Komponente/Komponente_K3SG2.csv'
K4_csv    = './Data/Komponente/Komponente_K4.csv'
K5_csv    = './Data/Komponente/Komponente_K5.csv'
K6_csv    = './Data/Komponente/Komponente_K6.csv'    

# vehicle data files
bestFahr1_11_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM1_Typ11.csv'
bestFahr1_12_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM1_Typ12.csv'
bestFahr2_21_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM2_Typ21.csv'
bestFahr2_22_csv = './Data/Fahrzeug/Bestandteile_Fahrzeuge_OEM2_Typ22.csv'

fahr1_11_csv = './Data/Fahrzeug/Fahrzeuge_OEM1_Typ11.csv'
fahr1_12_csv = './Data/Fahrzeug/Fahrzeuge_OEM1_Typ12.csv'
fahr2_21_csv = './Data/Fahrzeug/Fahrzeuge_OEM2_Typ21.csv'
fahr2_22_csv = './Data/Fahrzeug/Fahrzeuge_OEM2_Typ22.csv'

In [ ]:
# read and convert all the txt data files to csv
txt2csv(K1DI2_txt, K1DI2_csv, '	', '\\')
txt2csv(K2LE1_txt, K2LE1_csv, '', 'II')
txt2csv(K2LE2_txt, K2LE2_csv, '', '\\')
txt2csv(K2ST1_txt, K2ST1_csv, '', '|')
txt2csv(K3AG2_txt, K3AG2_csv, '', '\\')
txt2csv(K7_txt   , K7_csv   , '', '	')

In [189]:
# separate the data arrangements into 4 types, namely A, B, C, and D. 
# this separation is based on the column names of the datetime data type
A = ['Fehlerhaft_Datum', 'origin']
B = ['Produktionsdatum.x', 'Fehlerhaft_Datum.x', 
     'Produktionsdatum.y', 'Fehlerhaft_Datum.y']
C = ['Produktionsdatum.x', 'Fehlerhaft_Datum.x', 
     'Produktionsdatum.y', 'Fehlerhaft_Datum.y', 
     'Produktionsdatum', 'Fehlerhaft_Datum']
D = ['Produktionsdatum', 'Fehlerhaft_Datum']

# read the converted csv files
K1DI2 = pd.read_csv(K1DI2_csv, parse_dates=A, low_memory=False)
K2LE1 = pd.read_csv(K2LE1_csv, parse_dates=B, low_memory=False)
K2LE2 = pd.read_csv(K2LE2_csv, parse_dates=A, low_memory=False)
K2ST1 = pd.read_csv(K2ST1_csv, parse_dates=D, low_memory=False)
K3AG2 = pd.read_csv(K3AG2_csv, parse_dates=A, low_memory=False)
K7    = pd.read_csv(K7_csv,    parse_dates=A, low_memory=False)

# read the rest of the csv files
K1BE1 = pd.read_csv(K1BE1_csv, parse_dates=A, low_memory=False)
K1BE2 = pd.read_csv(K1BE2_csv, parse_dates=A, low_memory=False, sep=';')
K1DI1 = pd.read_csv(K1DI1_csv, parse_dates=C, low_memory=False)
K2ST2 = pd.read_csv(K2ST2_csv, parse_dates=A, low_memory=False, sep=';')
K3AG1 = pd.read_csv(K3AG1_csv, parse_dates=C, low_memory=False)
K3SG1 = pd.read_csv(K3SG1_csv, parse_dates=B, low_memory=False)
K3SG2 = pd.read_csv(K3SG2_csv, parse_dates=A, low_memory=False)
K4    = pd.read_csv(K4_csv,    parse_dates=B, low_memory=False, sep=';')
K5    = pd.read_csv(K5_csv,    parse_dates=B, low_memory=False)
K6    = pd.read_csv(K6_csv,    parse_dates=A, low_memory=False, sep=';')

# read the vehicle data:
bestFahr1_11 = pd.read_csv(bestFahr1_11_csv)
bestFahr1_12 = pd.read_csv(bestFahr1_12_csv)
bestFahr2_21 = pd.read_csv(bestFahr2_21_csv, sep=';')
bestFahr2_22 = pd.read_csv(bestFahr2_22_csv, sep=';')
fahr1_11 = pd.read_csv(fahr1_11_csv, parse_dates=D, low_memory=False)
fahr1_12 = pd.read_csv(fahr1_12_csv, parse_dates=D, low_memory=False)
fahr2_21 = pd.read_csv(fahr2_21_csv, parse_dates=A, low_memory=False)
fahr2_22 = pd.read_csv(fahr2_22_csv, parse_dates=A, low_memory=False, sep=';')

For datasets with data arrangements of type B and C, we need to consolidate the columns and eliminate the .x and .y suffixes. For type B, the tables are separated into 2, whereas for type C, the tables are separated into 3.

In [190]:
# column names to be renamed for type B
col_names_x = {'Produktionsdatum.x':'Produktionsdatum', 
               'Herstellernummer.x':'Herstellernummer',	
               'Werksnummer.x':'Werksnummer',
               'Fehlerhaft.x':'Fehlerhaft', 
               'Fehlerhaft_Datum.x':'Fehlerhaft_Datum',
               'Fehlerhaft_Fahrleistung.x':'Fehlerhaft_Fahrleistung'}
col_names_y = {'Produktionsdatum.y':'Produktionsdatum', 
               'Herstellernummer.y':'Herstellernummer',	
               'Werksnummer.y':'Werksnummer',
               'Fehlerhaft.y':'Fehlerhaft', 
               'Fehlerhaft_Datum.y':'Fehlerhaft_Datum',
               'Fehlerhaft_Fahrleistung.y':'Fehlerhaft_Fahrleistung'}

# separate type B tables based on the suffixes, rename the columns, and 
# concatenate vertically, and finally extract the columns that contain the data.
# the cleaned up dataset is added with suffix '_c'
K2LE1_x = K2LE1[K2LE1['ID_Sitze.x'].notna()].rename(columns=col_names_x)
K2LE1_y = K2LE1[K2LE1['ID_Sitze.y'].notna()].rename(columns=col_names_y)
K2LE1_x = K2LE1_x.rename(columns={'ID_Sitze.x':'ID_Sitze'})
K2LE1_y = K2LE1_y.rename(columns={'ID_Sitze.y':'ID_Sitze'})
K2LE1_x = K2LE1_x.loc[:,'ID_Sitze':'Fehlerhaft_Fahrleistung']
K2LE1_y = K2LE1_y.loc[:,'ID_Sitze':'Fehlerhaft_Fahrleistung']
K2LE1_c = pd.concat([K2LE1_x, K2LE1_y], axis=0).reset_index(drop=True)

K3SG1_x = K3SG1[K3SG1['ID_Schaltung.x'].notna()].rename(columns=col_names_x)
K3SG1_y = K3SG1[K3SG1['ID_Schaltung.y'].notna()].rename(columns=col_names_y)
K3SG1_x = K3SG1_x.rename(columns={'ID_Schaltung.x':'ID_Schaltung'})
K3SG1_y = K3SG1_y.rename(columns={'ID_Schaltung.y':'ID_Schaltung'})
K3SG1_x = K3SG1_x.loc[:,'ID_Schaltung':'Fehlerhaft_Fahrleistung']
K3SG1_y = K3SG1_y.loc[:,'ID_Schaltung':'Fehlerhaft_Fahrleistung']
K3SG1_c = pd.concat([K3SG1_x, K3SG1_y], axis=0).reset_index(drop=True)

K4_x = K4[K4['ID_Karosserie.x'].notna()].rename(columns=col_names_x)
K4_y = K4[K4['ID_Karosserie.y'].notna()].rename(columns=col_names_y)
K4_x = K4_x.rename(columns={'ID_Karosserie.x':'ID_Karosserie'})
K4_y = K4_y.rename(columns={'ID_Karosserie.y':'ID_Karosserie'})
K4_x = K4_x.loc[:,'ID_Karosserie':'Fehlerhaft_Fahrleistung']
K4_y = K4_y.loc[:,'ID_Karosserie':'Fehlerhaft_Fahrleistung']
K4_c = pd.concat([K4_x, K4_y], axis=0).reset_index(drop=True)

K5_x = K5[K5['ID_Karosserie.x'].notna()].rename(columns=col_names_x)
K5_y = K5[K5['ID_Karosserie.y'].notna()].rename(columns=col_names_y)
K5_x = K5_x.rename(columns={'ID_Karosserie.x':'ID_Karosserie'})
K5_y = K5_y.rename(columns={'ID_Karosserie.y':'ID_Karosserie'})
K5_x = K5_x.loc[:,'ID_Karosserie':'Fehlerhaft_Fahrleistung']
K5_y = K5_y.loc[:,'ID_Karosserie':'Fehlerhaft_Fahrleistung']
K5_c = pd.concat([K5_x, K5_y], axis=0).reset_index(drop=True)

# separate type C tables based on the suffixes, rename the columns, and 
# concatenate vertically, and finally extract the columns that contain the data.
# the cleaned up dataset is added with suffix '_c'
K1DI1_x = K1DI1[K1DI1['ID_Motor.x'].notna()].loc[:,'ID_Motor.x':'Fehlerhaft_Fahrleistung.x'].rename(columns=col_names_x)
K1DI1_y = K1DI1[K1DI1['ID_Motor.y'].notna()].loc[:,'ID_Motor.y':'Fehlerhaft_Fahrleistung.y'].rename(columns=col_names_y)
K1DI1_  = K1DI1[K1DI1['ID_Motor'].notna()]
K1DI1_x = K1DI1_x.rename(columns={'ID_Motor.x':'ID_Motor'})
K1DI1_y = K1DI1_y.rename(columns={'ID_Motor.y':'ID_Motor'})
K1DI1_  = K1DI1_.loc[:,'ID_Motor':'Fehlerhaft_Fahrleistung']
K1DI1_c = pd.concat([K1DI1_x, K1DI1_y, K1DI1_], axis=0).reset_index(drop=True)

Now with all the datasets clean and proper, we can start doing the case analysis. In this case study, the time period taken is between January 1st, 2011 and December 31st, 2015. The components chosen are components installed in the vechicles produced within the time period, which is based on the components list in the vehicle database. This is to ensure the same number of each component type within the time period.

First, we filter out the vehicle datasets, taking only data for vehicle produced during the time period. From the vehicle component dataset, we filter out the component datasets, taking only the components installed in the vehicles. Then we start finding out how many components installed in the vehicles experience failures, leading to eventual assessment of the suppliers.

For OEM2 (dataframe name 'fahr2_21' and 'fahr2_22'), the production date column is missing. So what we can do is create a column of production dates using the data from the 'Produktionsdatum_Origin_01011970' column. This column contains the number of days passed since January 1st, 1970. What we do is we add the number of days from the 'Produktionsdatum_Origin_01011970' column to the dates in the 'origin' column.

In [191]:
fahr2_21['Produktionsdatum'] = fahr2_21['origin'] + pd.to_timedelta(fahr2_21['Produktionsdatum_Origin_01011970'], unit='days')
fahr2_22['Produktionsdatum'] = fahr2_22['origin'] + pd.to_timedelta(fahr2_22['Produktionsdatum_Origin_01011970'], unit='days')

In [192]:
# prepare a dataset of vehicles produced from january 1st, 2011 until december 31st, 2015
fahr1_11_tp = (fahr1_11['Produktionsdatum'] >= '2011-01-01') & (fahr1_11['Produktionsdatum'] <= '2015-12-31')
fahr1_12_tp = (fahr1_12['Produktionsdatum'] >= '2011-01-01') & (fahr1_12['Produktionsdatum'] <= '2015-12-31')
fahr2_21_tp = (fahr2_21['Produktionsdatum'] >= '2011-01-01') & (fahr2_21['Produktionsdatum'] <= '2015-12-31')
fahr2_22_tp = (fahr2_22['Produktionsdatum'] >= '2011-01-01') & (fahr2_22['Produktionsdatum'] <= '2015-12-31')

fahr1_11_2011_15 = fahr1_11.loc[fahr1_11_tp,['ID_Fahrzeug','Produktionsdatum']].reset_index(drop=True)
fahr1_12_2011_15 = fahr1_12.loc[fahr1_12_tp,['ID_Fahrzeug','Produktionsdatum']].reset_index(drop=True)
fahr2_21_2011_15 = fahr2_21.loc[fahr2_21_tp,['ID_Fahrzeug','Produktionsdatum']].reset_index(drop=True)
fahr2_22_2011_15 = fahr2_22.loc[fahr2_22_tp,['ID_Fahrzeug','Produktionsdatum']].reset_index(drop=True)

In [194]:
pd.merge(fahr2_22_2011_15, bestFahr2_22.loc[:,bestFahr2_22.columns != 'X1'], on='ID_Fahrzeug')

,ID_Fahrzeug,Produktionsdatum,X1,ID_Karosserie,ID_Schaltung,ID_Sitze,ID_Motor
0,22-2-22-26,2012-08-09,141595,K7-114-1142-7,K3AG2-105-1051-65138,K2LE2-111-1111-47,K1BE2-104-1041-149431
1,22-2-22-147,2011-12-12,115545,K7-114-1142-136,K3AG2-105-1051-76,K2LE2-111-1111-164,K1DI2-102-1021-91861
2,22-2-22-482,2012-07-20,139371,K7-114-1142-513,K3AG2-105-1051-64224,K2ST2-109-1092-1365,K1DI2-102-1021-110174
3,22-2-22-483,2012-08-16,142411,K7-114-1142-515,K3AG2-105-1051-65470,K2ST2-109-1092-1367,K1BE2-104-1041-150301
4,22-2-22-548,2012-04-19,129250,K7-114-1142-813,K3SG2-105-1051-325512,K2ST2-109-1092-1466,K1BE2-101-1011-34240
...,...,...,...,...,...,...,...
192779,22-2-22-215467,2015-12-31,272411,K7-113-1132-119410,K3SG2-108-1081-172516,K2ST2-110-1101-188901,K1DI2-103-1031-143489
192780,22-2-22-215468,2015-12-31,272412,K7-113-1132-119411,K3SG2-108-1081-172524,K2ST2-110-1101-188904,K1DI2-103-1031-143490
192781,22-2-21-51921,2015-12-31,272413,K7-113-1132-119415,K3SG2-105-1051-288405,K2ST2-110-1101-188907,K1DI2-103-1031-143491
192782,22-2-21-51922,2015-12-31,272414,K7-113-1132-119418,K3SG2-105-1051-288430,K2ST2-110-1101-188909,K1DI2-103-1031-143492
